# Converts the English Wildlife data to chichewa using Google Translate

In [28]:
import pandas as pd
import requests
from typing import List, Dict, Tuple, Optional
from tqdm import tqdm
from googletrans import Translator
from bs4 import BeautifulSoup as bs
import re
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# Using readlines()
wildlife_df = pd.read_csv('Wildlife/wildlife.csv')

In [3]:
wildlife_df.shape

(64, 1)

In [4]:
wildlife_df.iloc[0]

Text    Elephants are not afraid to walk long distance...
Name: 0, dtype: object

In [5]:
data = pd.read_csv('Data/Train.csv')

In [6]:
data.head()

,ID,Text,Label
0,ID_AASHwXxg,Mwangonde: Khansala wachinyamata Akamati achi...,POLITICS
1,ID_AGoFySzn,MCP siidakhutire ndi kalembera Chipani cha Ma...,POLITICS
2,ID_AGrrkBGP,Bungwe la MANEPO Lapempha Boma Liganizire Anth...,HEALTH
3,ID_AIJeigeG,Ndale zogawanitsa miyambo zanyanya Si zachile...,POLITICS
4,ID_APMprMbV,Nanga wapolisi ataphofomoka? Masiku ano sichi...,LAW/ORDER


In [7]:
data['Text'][0]

' Mwangonde: Khansala wachinyamata Akamati achinyamata ndi atsogoleri a mawa, ambiri amaganiza kuti izi ndi nkhambakamwa chabe. Koma achinyamata ena, monga Lusubilo Mwangonde, akukwaniritsa akupherezetsa mawuwa osati pongolota kuti adzakhala, koma kutsogolera kumene chifukwa nthawi yawo yakwana. DAILES BANDA adacheza ndi Mwangonde, khansala wachinyama, yemwe akuimira Jumbo Ward mumzinda wa Mzuzu, motere:  Chisale watuluka nkumangidwanso  Sipakala waimitsa Nyumba ya Malamulo  Pa Wenela pasintha zedi Ali ndi masomphenya: Mwangonde Tikudziweni  Ndine Lusubilo Mwangonde, ndili ndi zaka 27 zakubadwa. Ndinabadwa mbanja la ana asanu ndipo ndine wachinayi kubadwa. Ndimachokera mmudzi mwa Mwamalopa, kwa Paramount Chief Kyungu mboma la Karonga. Sindili pabanja pakadalipano.\n Mbiri ya maphunziro anu ndi yotani? Maphunziro anga a pulaimale ndidachitira kusukula yapulaiveti ya Viphya mumzinda wa Mzuzu ndipo asekondale ndidachitira pa Phwezi Boys mboma la Rumphi. Ndili ndi diploma ya Accounting ndi

In [15]:
np.unique(data['Label'])

array(['ARTS AND CRAFTS', 'CULTURE', 'ECONOMY', 'EDUCATION', 'FARMING',
       'FLOODING', 'HEALTH', 'LAW/ORDER', 'LOCALCHIEFS', 'MUSIC',
       'OPINION/ESSAY', 'POLITICS', 'RELATIONSHIPS', 'RELIGION', 'SOCIAL',
       'SOCIAL ISSUES', 'SPORTS', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT',
       'WITCHCRAFT'], dtype=object)

In [16]:
data.loc[data['Label'] == 'WILDLIFE/ENVIRONMENT']

,ID,Text,Label
9,ID_AcAFvvEl,Alosera mvula kugwa mochuluka Nthambi yoona z...,WILDLIFE/ENVIRONMENT
13,ID_AlMentDU,"Chilengedwe chibwerere ku Mwanza Mapiri, nkha...",WILDLIFE/ENVIRONMENT
43,ID_CLayVWos,NGAMBA IOPSEZANSO CHAKA CHINOUNDUNA Chaka cha...,WILDLIFE/ENVIRONMENT
78,ID_DRCsFABS,MBS iletsa mafuta a mtimachubu Amalawi amene ...,WILDLIFE/ENVIRONMENT
85,ID_DWjlQjyO,Dziko la Bahamas Lilonjeza Kuthandiza Mzika Za...,WILDLIFE/ENVIRONMENT
108,ID_EIYNeYVJ,Njovu ipha mlenje Tsoka sasimba. Akanadziwa m...,WILDLIFE/ENVIRONMENT
276,ID_KezdVoQO,Akhristu Ali ndi Udindo Wosamalira Chilengedwe...,WILDLIFE/ENVIRONMENT
284,ID_KyNhavho,Lingalirani zobzala mitengo Mvula masiku ano ...,WILDLIFE/ENVIRONMENT
320,ID_MMoDCeVi,Mwambo wogwetsa simba Sukulu zatsekera ndipo ...,WILDLIFE/ENVIRONMENT
366,ID_OGXejpFI,K5 biliyoni yothana ndi matenda a mkungudza B...,WILDLIFE/ENVIRONMENT


In [17]:
from googletrans import Translator

In [18]:
def get_free_proxies():
    url = "https://free-proxy-list.net/"
    # get the HTTP response and construct soup object
    soup = bs(requests.get(url).content, "html.parser")
    proxies = []
    #print(soup)
    for row in soup.find("table", attrs={"class": "table table-striped table-bordered"}).find_all("tr")[1:]:
        tds = row.find_all("td")
        try:
            ip = tds[0].text.strip()
            port = tds[1].text.strip()
            host = f"{ip}:{port}"
            proxies.append(host)
        except IndexError:
            continue
    return proxies

In [19]:
translator = Translator(proxies=get_free_proxies())

In [20]:
translator.translate(data['Text'][0],src='ny',dest='en').text

"Mwangonde: A young councilor When they say young people are the leaders of tomorrow, many think that this is just a myth. But some young people, such as Lusubilo Mwangonde, are fulfilling this promise by not just pretending to be, but taking the lead because their time has come. DAILES BANDA spoke to Mwangonde, a wildlife councilor representing Jumbo Ward in Mzuzu, as follows: Chisale has been re-arrested Sipakala has suspended Parliament in Wenela I was born in a family of five children and am the fourth born. I come from Mwamalopa village, Paramount Chief Kyungu in Karonga district. I'm not married yet.\n What is your educational background? My primary education was at Viphya Private School in Mzuzu and my high school education was at Phwezi Boys' Rumphi District. I have a Diploma in Accounting and am currently pursuing a degree and Chartered Accounting at the Malawi College of Accountancy (MCA).\n How did you get into politics? Ever since I was a teenager, 12 years old, I have been

In [21]:
for i,line in enumerate(wildlife_df['Text']):
    print(line)
    if(i>3):
        break

Elephants are not afraid to walk long distances. In 2016, one elephant walked nearly 130 miles from Kenya to Somalia in just three weeks. In 2020, 15 elephants in China set off on a 310-mile journey, capturing global attention.Considering these viral migrations, it is fitting that 16-year-old Cole Culver’s plan to raise awareness for elephants involved a long journey by foot.Last year, Mike Culver encouraged his son Cole to find a way to give back. Some people suggested that Cole, who is autistic, should do something to benefit autism. Mike pushed him to think about others instead.I think it’s very important for him to understand that the world does not revolve around our own issues, but around everyone’s issues, said Mike.Cole knew who he wanted to help: elephants, his favorite animal. After doing some research, Cole and his family found African Wildlife Foundation. They wanted to ensure that the money they raised was not focused narrowly on wildlife but was helping people — through h

In [22]:
li_chi = []
for i,phrase in enumerate(wildlife_df['Text']):
    li_chi.append(translator.translate(phrase,src='en',dest='ny').text)
    #break

In [23]:
len(li_chi)

64

In [24]:
li_chi[55]

"Kodi ndi chiyani chomwe chimatsogolera asilikali olimba mtima awa pankhondo yaku Africa yolimbana ndi kusaka nyama? Kaŵirikaŵiri, amasonkhezeredwa ndi chikondi chozama cha nyama zakuthengo ndi madera akuthengo. Ife, anthu amene timagwira ntchito pano, ntchitoyi ili m’magazi mwathu, akutero Philbert Mwasi, wofufuza pa LUMO Wildlife Conservancy ya Kenya. Ngati titapuma pa ntchito yathu, sitingakhale pamtendere ndi ife tokha. Ndimakonda kwambiri ntchito ya alonda, anatero wofufuza wina wa LUMO, Ludovika Malemba. Ndinali wofufuza zamasewera pamene ndinali kusukulu ya pulayimale— kuteteza kuli m’magazi mwanga. Ngakhale kuti chiwopsezo ndi zovuta zili mbali ya kufotokozera kwa ntchito, alonda ku Africa konse ali ndi zinthu zochepa, makamaka poyerekeza ndi opha nyama omwe amagwira ntchito m'malo ozembetsa nyama zakuthengo padziko lonse lapansi. M'mikangano yambiri, timakumana ndi zida zankhondo, ophunzitsidwa bwino, komanso opha nyama mwadongosolo - omwe ambiri mwa iwo ali ndi usilikali, ate

In [25]:
df_wildlife = pd.DataFrame(li_chi, columns =['Text'])
df_wildlife['Label'] = 'WILDLIFE/ENVIRONMENT'
#df_flood['ID'] = [i for i in range(len(li_chi))]

In [26]:
df_wildlife.head()

,Text,Label
0,Njovu siziwopa kuyenda mitunda yaitali. Mu 201...,WILDLIFE/ENVIRONMENT
1,"Belise Kariza, Mtsogoleri wa Dziko la African ...",WILDLIFE/ENVIRONMENT
2,Kuyesetsa kwambiri kuteteza gorila m'zaka maku...,WILDLIFE/ENVIRONMENT
3,Pansi pa nthaka ndi pansi pa dziko lapansi pal...,WILDLIFE/ENVIRONMENT
4,Akugwira ntchito yotunga madzi m'mawa uliwonse...,WILDLIFE/ENVIRONMENT


In [27]:
df_wildlife.to_csv('Wildlife/wildlife_chichewa.csv',index=False)